# Module 8 — Agents & Automation
**Controlled Decision-Making with LLMs**

This notebook demonstrates **bounded, safe agents**.
Agents decide, act, and stop — they do not run autonomously.

## Core Principle

An agent is a system that uses an LLM to choose between *pre-approved actions*.

- LLM proposes
- Code validates
- System executes

## 8.1 Pipelines vs Agents

Pipelines are fixed; agents introduce controlled decision points.

## 8.2 Simplest Agent Shape

In [ ]:
def simple_agent(input_text):
    decision = decide_action(input_text)
    if decision == "ACTION_A":
        return "Did action A"
    if decision == "ACTION_B":
        return "Did action B"
    return "Refused"

## 8.3 Allowed Actions

In [ ]:
ALLOWED_ACTIONS = {
    "retrieve_and_answer",
    "classify_only",
    "refuse"
}

## Mock LLM Client

In [ ]:
class MockLLMClient:
    def chat(self, prompt):
        if "rate" in prompt.lower():
            return '{"action": "retrieve_and_answer"}'
        if "classify" in prompt.lower():
            return '{"action": "classify_only"}'
        return '{"action": "refuse"}'

llm_client = MockLLMClient()

## 8.4 Decision Step

In [ ]:
import json

def decide_action(llm_client, user_input):
    prompt = f'''
Choose exactly one action:
- retrieve_and_answer
- classify_only
- refuse

User input:
{user_input}

Return ONLY JSON:
{{ "action": "<action>" }}
'''
    raw = llm_client.chat(prompt)
    action = json.loads(raw).get("action")
    if action not in ALLOWED_ACTIONS:
        return "refuse"
    return action

## 8.5 Classification Tool

In [ ]:
def classify_only(text):
    if "rate" in text.lower():
        return "Topic: Interest Rates"
    if "loan" in text.lower():
        return "Topic: Lending"
    return "Topic: General"

## 8.6 Retrieval + Answer Tool (Mock RAG)

In [ ]:
def retrieve_and_answer(question):
    return "The central bank raised rates to control inflation."

## 8.7 Refusal Tool

In [ ]:
def refuse():
    return "I am not authorised to act on this request."

## 8.8 Assemble the Triage Agent

In [ ]:
def triage_agent(user_input):
    action = decide_action(llm_client, user_input)
    if action == "retrieve_and_answer":
        return retrieve_and_answer(user_input)
    if action == "classify_only":
        return classify_only(user_input)
    return refuse()

## 8.9 Run the Agent

In [ ]:
triage_agent("Why did the central bank raise rates?")

In [ ]:
triage_agent("Classify this sentence about loans")

In [ ]:
triage_agent("Should we approve a loan for customer X?")

## 8.10 Controlled Loop

In [ ]:
def agent_loop(user_input, max_steps=1):
    for _ in range(max_steps):
        return triage_agent(user_input)
    return "Agent stopped."

## 8.11 Infinite Loops Are Dangerous

In [ ]:
# Never allow unbounded loops with LLM calls
pass

## 8.12 Guardrails as Code

In [ ]:
def enforce_policy(action):
    return action in ALLOWED_ACTIONS

## 8.13 Logging Agent Decisions

In [ ]:
def log_agent_step(input_text, action, output):
    print({"input": input_text, "action": action, "output": output})

## 8.14 Failure Mode Demo

In [ ]:
triage_agent("Tell me about the game")

## 8.15 When NOT to Use Agents

Do not use agents for irreversible or financial decisions.
Bounded automation is safer than autonomy.

## Module Summary

You have built a bounded, auditable agent.
This prepares you for the capstone, where judgement matters.